### 1. Autoencoder 기반 이상 탐지 (LSTM/Transformer 등)   
1. 기본 개념   
정상(전문가) 데이터만으로 Autoencoder를 학습합니다.   
모델이 시퀀스(예: 30프레임 × 관절 24개)를 입력받아, 동일한 형태(30×24)를 복원하도록 학습합니다.   
정상 패턴을 충분히 학습한 후, 비정상 패턴이 들어오면 복원 오차가 커지게 됩니다.   
오차(예: MSE)가 특정 Threshold를 초과하면 “이상치”로 판단할 수 있습니다.   

2. 장점   
정상 데이터만 있으면 학습 가능하므로, 비정상 데이터가 적어도 어느 정도 동작합니다.   
시계열 특성을 반영해 LSTM Autoencoder나 Transformer Autoencoder를 사용하면, 러닝 동작의 연속적 특징을 잘 포착할 수 있습니다.   

3. 단점 / 고려사항   
데이터가 지속적으로 추가될 때, Autoencoder를 점진적으로 학습(Incremental Learning) 하는 것은 쉽지 않습니다.   
보통은 새로운 데이터(정상·비정상)를 모아서 주기적으로 재학습(fine-tuning)하거나, 전량 학습을 다시 하는 식으로 운영합니다.   


### 2. 지도학습(이진 분류) 모델 (딥러닝)
1. 기본 개념
정상과 비정상 모두 충분한 데이터를 확보하면, 이진 분류 모델을 직접 학습시킬 수 있습니다.   
시퀀스(30×24) 입력 → LSTM/GRU/Transformer 기반 네트워크 → “정상(1) vs 비정상(0)” 확률 출력   
비정상 데이터가 늘어날수록, 모델이 더 정확히 “나쁜 자세”를 인식할 수 있습니다.   

2. 장점   
정상/비정상 데이터가 충분하면, Autoencoder보다 직관적으로 “어떤 자세가 나쁜지”를 배울 수 있습니다.   
비정상 패턴이 다양해질수록 모델이 더 견고해집니다.   

3. 단점 / 고려사항   
비정상 데이터가 많아야 제대로 학습할 수 있습니다.   
추가 학습(Incremental Learning)을 구현하려면, 기존 모델에 새 데이터를 미니배치로 넣어 재학습하거나, 주기적으로 전량 학습을 다시 해야 합니다.   


### 3. 장기적인 모델 업데이트 전략
1. 데이터 저장 & 주기적 재학습   
새로운 사용자 데이터를 지속적으로 수집(정상·비정상 레이블)   
일정 주기(예: 한 달, 분기)마다 모델을 전량 재학습하거나, 파인튜닝하여 성능 개선   
딥러닝 프레임워크(TensorFlow/PyTorch)로 구현 가능   

2. 온라인 학습(Online/Incremental Learning)   
일부 알고리즘(예: 일부 계열의 Neural Network, 일부 scikit-learn 모델)은 부분 학습(Partial Fit)을 지원   
하지만 시계열 딥러닝 모델(LSTM, Transformer 등)은 아직 안정적인 온라인 학습 구현이 쉽지 않습니다.   
보통은 주기적 미니배치 재학습 형태가 현실적입니다.   

3. 데이터 품질 관리   
비정상 자세가 다양해질수록 모델이 학습해야 할 패턴이 늘어납니다.   
라벨링(“이 시퀀스는 나쁜 자세”) 품질이 높아야 모델도 안정적으로 성능을 높일 수 있습니다.   


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense

def build_lstm_autoencoder(timesteps, n_features, latent_dim=64):
    """
    LSTM Autoencoder를 구성하여 반환하는 함수.
    timesteps: 시퀀스 길이 (예: 30)
    n_features: 프레임당 피처 수 (예: 132 또는 관절 선택 후 24 등)
    latent_dim: 인코더의 LSTM 유닛 수 (자유롭게 조정 가능)
    """
    # 인코더
    inputs = Input(shape=(timesteps, n_features))
    encoded = LSTM(latent_dim)(inputs)
    
    # RepeatVector: 인코더 출력(1개 타임스텝)을 디코더 입력(timesteps개 타임스텝)으로 확장
    encoded = RepeatVector(timesteps)(encoded)
    
    # 디코더
    decoded = LSTM(latent_dim, return_sequences=True)(encoded)
    # 각 타임스텝마다 n_features를 출력해야 하므로 TimeDistributed(Dense(n_features))
    outputs = TimeDistributed(Dense(n_features))(decoded)
    
    autoencoder = Model(inputs, outputs)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


In [3]:
import pandas as pd
import numpy as np

def load_csv_and_make_sequences(csv_path, seq_length=30):
    df = pd.read_csv(csv_path)
    df = df.dropna()

    # 'frame' 열 제외 -> 33개 랜드마크 × 4개 값 = 132개 열
    data = df.iloc[:, 1:].values  # shape: (프레임 수, 132)
    n_frames = data.shape[0]

    sequences = []
    for i in range(n_frames - seq_length + 1):
        seq = data[i:i+seq_length]
        sequences.append(seq)
    return np.array(sequences)

# (선택) 특정 관절만 추리는 예시 (원한다면 사용):
def select_important_joints(frame_1d):
    """
    필요한 관절 인덱스만 추출.
    frame_1d.shape = (132,)
    """
    # 예: 왼엉덩이(23), 오른엉덩이(24), 왼무릎(25), 오른무릎(26), 왼발목(27), 오른발목(28)
    # important_landmarks = [23, 24, 25, 26, 27, 28]
    # ...
    return frame_1d  # 여기서는 전체 사용 (필요시 수정)

def load_csv_and_make_sequences_selected(csv_path, seq_length=30):
    df = pd.read_csv(csv_path)
    df = df.dropna()

    data = df.iloc[:, 1:].values  # (프레임 수, 132)
    n_frames = data.shape[0]

    sequences = []
    for i in range(n_frames - seq_length + 1):
        seq = data[i:i+seq_length]
        # 관절 선택
        seq_selected = np.array([select_important_joints(frame) for frame in seq])
        sequences.append(seq_selected)
    return np.array(sequences)


In [4]:
# 1) CSV 데이터 로드
csv_path = r"C:\barunrun\pose_data\pose_data1.csv"  # 실제 경로/파일명으로 변경
X_expert = load_csv_and_make_sequences(csv_path, seq_length=30)

print("X_expert shape:", X_expert.shape)  # 예: (545, 30, 132)

# 2) LSTM Autoencoder 모델 생성
timesteps = 30
n_features = X_expert.shape[2]  # 예: 132 (또는 관절 선택 시 24 등)
latent_dim = 64  # 임의 설정 가능

autoencoder = build_lstm_autoencoder(timesteps, n_features, latent_dim)
autoencoder.summary()

# 3) 모델 학습
# Autoencoder는 "입력=X_expert, 출력=X_expert" 형태로 학습
history = autoencoder.fit(
    X_expert, X_expert,
    epochs=20,
    batch_size=16,
    validation_split=0.1
)


X_expert shape: (545, 30, 132)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 132)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        50,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 132)        │         8,580 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,036 (359.52 KB)

 Trainable params: 92,036 (359.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.1670 - val_loss: 0.0116
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0078 - val_loss: 0.0058
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0041 - val_loss: 0.0052
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0036 - val_loss: 0.0049
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0034 - val_loss: 0.0047
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0032 - val_loss: 0.0047
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0032 - val_loss: 0.0047
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 10/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0031 - val_loss: 0.0046
Epoch 11/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0031 - val_loss: 0.0047
Epoch 12/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0

In [6]:
# 비정상 CSV 로드
csv_path_abnormal = r"C:\barunrun\pose_data\pose_data_abnormal1.csv"
X_abnormal = load_csv_and_make_sequences(csv_path_abnormal, seq_length=30)
print("X_abnormal shape:", X_abnormal.shape)

# 복원
X_abnormal_pred = autoencoder.predict(X_abnormal)

# MSE 계산
mse_abnormal = np.mean((X_abnormal - X_abnormal_pred)**2, axis=(1,2))

# 임계값 설정 (전문가 데이터로 계산)
X_expert_pred = autoencoder.predict(X_expert)
mse_expert = np.mean((X_expert - X_expert_pred)**2, axis=(1,2))
mean_mse_expert = np.mean(mse_expert)
std_mse_expert = np.std(mse_expert)
threshold = mean_mse_expert + 3*std_mse_expert  # 예시

# 비정상 중 이상치로 판단되는 개수
abnormal_detected = np.sum(mse_abnormal > threshold)
abnormal_ratio = abnormal_detected / len(mse_abnormal)
print(f"비정상 데이터 중 이상치로 판단된 시퀀스: {abnormal_detected}/{len(mse_abnormal)}")
print(f"비정상 판단 비율: {abnormal_ratio:.3f}")
print(f"임계값: {threshold:.5f}, 평균 MSE(전문가): {mean_mse_expert:.5f}, 표준편차: {std_mse_expert:.5f}")


X_abnormal shape: (381, 30, 132)
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
비정상 데이터 중 이상치로 판단된 시퀀스: 381/381
비정상 판단 비율: 1.000
임계값: 0.00634, 평균 MSE(전문가): 0.00312, 표준편차: 0.00107


In [7]:
csv_path_abnormal = r"C:\barunrun\pose_data\pose_data_abnormal1.csv"
X_abnormal = load_csv_and_make_sequences(csv_path_abnormal, seq_length=30)
print("X_abnormal shape:", X_abnormal.shape)

# Autoencoder로 복원
X_abnormal_pred = autoencoder.predict(X_abnormal)

# 복원 오차(MSE) 계산
mse_abnormal = np.mean((X_abnormal - X_abnormal_pred)**2, axis=(1,2))
print("비정상 데이터 MSE:", mse_abnormal[:10])  # 앞 10개 예시 출력

# 임계값 설정 (전문가 데이터에서 계산했던 mean_mse_expert, std_mse_expert가 있다고 가정)
threshold = mean_mse_expert + 3*std_mse_expert
abnormal_detected = np.sum(mse_abnormal > threshold)
abnormal_ratio = abnormal_detected / len(mse_abnormal)

print(f"비정상 데이터 중 이상치로 판단된 시퀀스: {abnormal_detected}/{len(mse_abnormal)}")
print(f"비정상 판단 비율: {abnormal_ratio:.3f}")
print(f"임계값: {threshold:.5f}, 평균 MSE(전문가): {mean_mse_expert:.5f}, 표준편차: {std_mse_expert:.5f}")


X_abnormal shape: (381, 30, 132)
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
비정상 데이터 MSE: [0.08844079 0.08927128 0.09007889 0.09065552 0.09124724 0.09181806
 0.09251252 0.09290484 0.09311747 0.09347493]
비정상 데이터 중 이상치로 판단된 시퀀스: 381/381
비정상 판단 비율: 1.000
임계값: 0.00634, 평균 MSE(전문가): 0.00312, 표준편차: 0.00107


In [8]:
##############################
# (1) 비정상 CSV 로드
##############################
csv_path_abnormal = r"C:\barunrun\pose_data\pose_data_abnormal1.csv"
X_abnormal = load_csv_and_make_sequences(csv_path_abnormal, seq_length=30)
print(f"[INFO] 비정상 데이터 로드 완료: {csv_path_abnormal}")
print("X_abnormal shape:", X_abnormal.shape)

##############################
# (2) Autoencoder 예측
##############################
X_abnormal_pred = autoencoder.predict(X_abnormal)

##############################
# (3) 복원 오차(MSE) 계산
##############################
mse_abnormal = np.mean((X_abnormal - X_abnormal_pred)**2, axis=(1,2))
print(f"[INFO] 비정상 데이터 MSE (처음 10개): {mse_abnormal[:10]}")

##############################
# (4) 임계값 설정
##############################
# 이미 전문가(정상) 데이터로부터 계산된 mean_mse_expert, std_mse_expert가 있다고 가정
# 예) mean_mse_expert = np.mean(mse_expert)
#     std_mse_expert = np.std(mse_expert)

threshold = mean_mse_expert + 3 * std_mse_expert  # 예시로 평균+3*표준편차 사용

##############################
# (5) 비정상 시퀀스 중 이상치 판단
##############################
abnormal_detected = np.sum(mse_abnormal > threshold)
abnormal_ratio = abnormal_detected / len(mse_abnormal)

##############################
# (6) 결과 출력
##############################
print("\n===== 비정상 데이터 예측 결과 =====")
print(f"총 {len(mse_abnormal)}개 시퀀스 중, 임계값 초과(이상치): {abnormal_detected}")
print(f"비정상 판단 비율: {abnormal_ratio:.3f}")
print(f"임계값: {threshold:.5f}")
print(f"전문가 데이터 평균 MSE: {mean_mse_expert:.5f}, 표준편차: {std_mse_expert:.5f}")


[INFO] 비정상 데이터 로드 완료: C:\barunrun\pose_data\pose_data_abnormal1.csv
X_abnormal shape: (381, 30, 132)
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
[INFO] 비정상 데이터 MSE (처음 10개): [0.08844079 0.08927128 0.09007889 0.09065552 0.09124724 0.09181806
 0.09251252 0.09290484 0.09311747 0.09347493]

===== 비정상 데이터 예측 결과 =====
총 381개 시퀀스 중, 임계값 초과(이상치): 381
비정상 판단 비율: 1.000
임계값: 0.00634
전문가 데이터 평균 MSE: 0.00312, 표준편차: 0.00107
